In [80]:
!pip3 install pandas


In [81]:
import os
import re
import pandas as pd
import shutil
import json
import difflib
import pprint as pp


In [82]:
### Funciones utiles para manejar archivos y directorios

In [83]:
def raiz():
    return '/'.join(os.getcwd().split('/')[:-1])


def ubicar(comision, *camino):
    ''' Ubica una carpeta en la comisión indicada '''
    carpeta = f'lab4-C{comision}' if comision else 'lab4' # Si la comisión es 0, se asume que es la catedra 
    comision = f"{raiz()}/{carpeta}"
    return '/'.join([comision, *camino])


def carpeta_alumno(alumno):
    ''' Genera el nombre de la carpeta del alumno '''
    return f"{alumno['orden']:02} - {alumno['legajo']} - {alumno['apellido']}, {alumno['nombre']}".strip()


def listar_carpetas(origen):
    lista = sorted(os.listdir(origen))
    lista = [c for c in lista if not c.startswith('.')]
    return lista


def ubicar_alummo(legajo, *camino, alumnos=None):
    ''' Dado el legajo de un alumno, devuelve la carpeta donde debería estar '''
    if alumnos is None: alumnos = cargar_alumnos()

    alumno = alumnos.loc[alumnos['legajo'] == str(legajo)].iloc[0]
    comision = int(alumno['comision'])
    return ubicar(comision, 'practicos', carpeta_alumno(alumno), *camino).strip()


def copiar_archivos(origen, destino, forzar=False):
    ''' Copia los archivos de una carpeta a otra '''

    # Verificar si la carpeta de destino existe, si no, crearla
    if not os.path.exists(destino):
        os.makedirs(destino)
    
    # Iterar sobre todos los archivos en la carpeta de origen
    for archivo in os.listdir(origen):
        ruta_origen  = os.path.join(origen, archivo)
        ruta_destino = os.path.join(destino, archivo)
        
        # Verificar si es un archivo y no un directorio
        if os.path.isfile(ruta_origen):
            if not os.path.exists(ruta_destino) or forzar:
                shutil.copy(ruta_origen, ruta_destino)


def renombrar_carpeta(origen, destino):
    ''' Renombra una carpeta '''

    # Crear el directorio de destino si no existe
    if not os.path.exists(destino):
        os.makedirs(destino)

    # Mover el contenido del directorio
    for archivo in os.listdir(origen):
        ruta_origen  = os.path.join(origen, archivo)
        ruta_destino = os.path.join(destino, archivo)

        # Borra el destino si ya existe
        if os.path.exists(ruta_destino):
            if os.path.isdir(ruta_destino):
                shutil.rmtree(ruta_destino)
            else:
                os.remove(ruta_destino)
        
        # Mueve los archivos
        shutil.move(ruta_origen, ruta_destino)
    
    # Eliminar el directorio fuente si está vacío
    if not os.listdir(origen): os.rmdir(origen)

### Cargar alumnos

Toma los alumnos que estan en alumnos.md y lo carga en un datatable.

El formato del archivo debe ser
```
## Comision N
1. Legajo Apellido, Nombre
```

In [84]:

def cargar_alumos(origen = 'alumnos.md'):
    ''' Carga los alumnos desde el archivo de texto '''

    with open(origen, 'r') as file:
        lineas = file.readlines()
    
    # Inicializar listas para almacenar los datos
    comisiones, ordenes, legajos, apellidos, nombres = [] ,[], [], [], []
    comision = None
    
    for linea in lineas:
        linea = re.sub(r'\|.*|^\s*|\s*$', '', linea) # Limpiar la línea
        linea = linea[:60]
        if linea.startswith('## Comisión'):   # Extraer el número de la comisión
            comision = int(linea.split(' ')[-1])
        elif linea and linea[0].isdigit():     # Extraer el número de orden, legajo, apellido y nombre
            match = re.match(r'(\d+)\.\s+(\d+)\s+([^,]+),\s+(.+)(|>.*)?', linea)
            if match: 
                comisiones.append(comision)
                ordenes.append(int(match.group(1)))
                legajos.append(match.group(2))
                apellidos.append(match.group(3))
                nombres.append(match.group(4))
    
    # Crear el DataFrame
    data = { 'comision': comisiones, 'orden': ordenes, 'legajo': legajos, 'apellido': apellidos, 'nombre': nombres}
    return pd.DataFrame(data)

def guardar_alumnos(alumnos, destino='alumnos.md', asistencias=None):
    salida = []
    salida.append('# Alumnos Lab4.2024')

    for comision in alumnos['comision'].unique():
        salida.append("")
        salida.append(f'## Comisión {comision}')
        for _, alumno in alumnos[alumnos['comision'] == comision].iterrows():
            asistio = ''
            if asistencias and alumno['legajo'] in asistencias:
                for practico, resultado in asistencias[alumno['legajo']].items():
                    if resultado['ausente']:
                        asistio += '🔴'
                    elif resultado['revisar']:
                        asistio += '🔵'
                    else:
                        asistio += '🟢'

            salida.append(f"{alumno['orden']:2}. {alumno['legajo']:5}  {alumno['apellido'] +", " + alumno['nombre']:50} {asistio}")

    with open(destino, 'w') as file:
        file.writelines("\n".join( salida))

print(cargar_alumos())

     comision  orden legajo        apellido  \
0           2      1  58764           Acuña   
1           2      2  59277         Aguirre   
2           2      3  58827  Albornoz Silva   
3           2      4  58952         Almiron   
4           2      5  58731         Alvarez   
..        ...    ...    ...             ...   
140         9     17  59074         Teseyra   
141         9     18  59056       Villafañe   
142         9     19  59314          Yapura   
143         9     20  59186          Roldan   
144         9     21  59055         Saravia   

                                          nombre  
0    Ana Sofía                                    
1      Emanuel                                    
2             Alejo Miguel                        
3      Maicol Leonel                              
4      Nicolás                                    
..                                           ...  
140     Juan Ignacio                              
141       Lucas Gastón     

### Detectar datos duplicados

Revisa si los legajos estan duplicados (error), y como curiosidad si hay nombre y apellidos repetidos.

In [85]:
def detectar_duplicados(data):

    def duplicados(columna):
        return data[data[columna].duplicated(keep=False)].sort_values(by=columna)
    
    legajos = duplicados('legajo')
    
    print("Registros con legajo duplicado:")
    if len(legajos) == 0:
        print("> No hay legajos duplicados")
    else:
        print("> ERROR: Se encontraron legajos duplicados")
        print(legajos)

    # Listar registros con nombre duplicado
    nombres = duplicados('nombre')
    num_nombres = len(nombres['nombre'].unique())
    if num_nombres: 
        print(f"\nHay {num_nombres} NOMBRES duplicados en {len(nombres)} registros")
        print(nombres)    
        
    # Listar registros con apellido duplicado
    apellidos = duplicados('apellido')
    num_apellidos = len(apellidos['apellido'].unique())
    if num_apellidos:
        print(f"\nHay {num_apellidos} APELLIDOS duplicados en {len(apellidos)} registros")
        print(apellidos)

detectar_duplicados(cargar_alumos())

Registros con legajo duplicado:
> No hay legajos duplicados

Hay 13 APELLIDOS duplicados en 29 registros
     comision  orden legajo     apellido  \
4           2      5  58731      Alvarez   
5           2      6  58828      Alvarez   
6           2      7  59176    Argañaraz   
7           2      8  58909    Argañaraz   
38          5      2  58876         Díaz   
37          5      1  58690         Díaz   
47          5     11  58740       García   
46          5     10  59154       García   
53          5     17  59068     Gonzalez   
52          5     16  58720     Gonzalez   
21          2     22  55906     González   
20          2     21  55533     González   
51          5     15  59488     González   
19          2     20  58832     González   
56          5     20  58761       Juarez   
57          5     21  58758       Juarez   
62          5     26  58727      Lazarte   
61          5     25  58756      Lazarte   
102         7     12  59194       Medina   
68          5  

### Generar trabajo prácticos
Copia de la carpeta prácticos el tp a cada una de los repositorios 


In [86]:

def publicar(practico):
    alumnos = cargar_alumos()

    # Copia los archivos del práctico a cada alumno
    for legajo in alumnos['legajo']:
        destino = f"{ubicar_alummo(legajo, alumnos=alumnos)}/{practico}" 
        copiar_archivos(f'./practicos/{practico}', destino)
        
    # Copia el enunciados del práctico a cada comisión
    for comision in [2,5,7,9]:
        destino = ubicar(comision)
        shutil.copy(f'./practicos/enunciados/{practico}.md', f"{destino}/README.md")

publicar('tp1')

### Normalizar nombres de carpetas.
Esta función permite revisar que los nombres estén bien escritos (tomando en cuenta el número de legajo).

In [87]:
def extraer_legajo(carpeta):
    ''' Extrae el legajo del nombre de la carpeta '''
    m = re.match(r'.*(\d{5}).*', carpeta)
    if m: return m.group(1)
    return None

def normalizar():
    alumnos = cargar_alumos()

    for comision in [2, 5 ,7, 9]:
        # listar todas las carpetas de alumnos que esten en la carpeta de la comisión

        carpetas = listar_carpetas(ubicar(comision, 'practicos'))
        for alumno in carpetas:
            legajo = extraer_legajo(alumno)
            if not legajo: continue
            
            origen  = ubicar(comision, 'practicos', alumno)
            destino = ubicar_alummo(legajo, alumnos=alumnos)
            
            if origen == destino: continue
            print(f"Renombrando [{origen}]\n            [{destino}]")

            renombrar_carpeta(origen, destino)
            
normalizar()


### Soluciona el proble que publique TP1 directamente en la raiz

In [88]:
def mover_ejercicio_a_tp1():
    alumnos = cargar_alumos()
    for comision in [2,5,7,9]:
        # listar todas las carpetas de alumnos que esten en la carpeta de la comisión
        carpetas = os.listdir(ubicar(comision, 'practicos'))
        for alumno in carpetas:
            alumno = alumno.strip()
            m = re.match(r'\d{2}\s+-?\s*(\d+) - ([^,]+), (.+)', alumno)
            if not m: continue

            legajo = m.group(1)
            origen = ubicar_alummo(legajo, alumnos=alumnos)

            archivos = os.listdir(origen)
            if '.DS_Store' in archivos: archivos.remove('.DS_Store')

            archivos.sort()
            for archivo in archivos:
                print(f"{origen}/{archivo}")
                ruta_origen = f"{origen}/{archivo}"
                ruta_destino = f"{origen}/tp1/{archivo}"

                if not os.path.exists(f"{origen}/tp1"):
                    os.makedirs(f"{origen}/tp1")
                    shutil.move(ruta_origen, ruta_destino)
                
# mover_ejercicio_a_tp1()

### Funciones para comparar dos Jupiter Notebook 

In [89]:
def cargar_cuaderno(origen):
    ''' Dado un archivo .ipynb, devuelve una lista con las líneas de código 
        de cada celda de código en el cuaderno.'''
    
    def limpiar(origen): # Elimina líneas en blanco y comentarios
        lineas = origen if isinstance(origen, list) else origen.splitlines()
        lineas = [linea.strip() for linea in lineas]
        return [linea for linea in lineas if linea and not linea.startswith('#')]
    
    def codigo(cuaderno): # Extrae las líneas de código de las celdas de código
        celdas = cuaderno.get('cells', [])
        celdas = [celda for celda in celdas if celda.get('cell_type') == 'code']
        return [limpiar(celda.get('source', [])) for celda in celdas]
    
    with open(origen, 'r', encoding='utf-8') as f:
        cuaderno = json.load(f)

    return codigo(cuaderno)


def comparar_cuadernos(original, modificado):
    ''' Dados dos cuadernos de Jupyter, devuelve la cantidad 
        de líneas de código que se agregaron en el segundo 
        respecto del primero.'''
    
    def igualar(a, b): # Ajusta las listas a y b para que tengan la misma longitud
        while len(a) < len(b): a.append([])
        while len(b) < len(a): b.append([])
        return zip(a, b)    
    
    def cambios(a, b): # Cuenta las líneas que se agregaron en b respecto de a
        diferencias = difflib.unified_diff(a, b, lineterm='')
        diferencias = [l for l in diferencias if l.startswith('+') and not l.startswith('+++')]
        return len(diferencias)
    
    conteo = [cambios(o, m) for o, m in igualar(original, modificado)]

    return {'diferencias': conteo, 'total': sum(conteo)}


In [90]:
def listar_cuadernos(*origen):
    ''' Dada una carpeta, devuelve una lista con los cuadernos de Jupyter (.ipynb) que contiene.'''
    archivos = listar_carpetas(*origen)
    return [a for a in archivos if a.endswith('.ipynb')]

# def analizar(practico, comisiones=[2,5,7,9]):
#     if not isinstance(comisiones, list): comisiones = [comisiones]

#     cuadernos = listar_cuadernos(ubicar(0, 'practicos', practico))
#     print(f"Analizando {practico}...")
#     print(cuadernos)

# analizar('tp1')
# analizar('tp2')


In [92]:

def analizar_cambios(practicos, comisiones=[2,5,7,9], resultados={}):
    if not isinstance(comisiones, list): comisiones = [comisiones]
    if not isinstance(practicos, list):  practicos = [practicos]

    normalizar()

    for practico in practicos:
        archivos = listar_cuadernos(ubicar(0, 'practicos', practico))
        
        for archivo in archivos:
            ruta_original = ubicar(0,'practicos', practico, archivo)
            print(f"Analizando {ruta_original}:")
            original = cargar_cuaderno(ruta_original)
            celdas = len(original)

            for comision in comisiones:
                carpetas = listar_carpetas(ubicar(comision, 'practicos'))
                for alumno in carpetas:
                    destino = ubicar(comision, 'practicos', alumno, practico, archivo)

                    con_cambios = cargar_cuaderno(destino)
                    cambios     = comparar_cuadernos(original, con_cambios)

                    legajo = extraer_legajo(alumno)
                    if not legajo in resultados: resultados[legajo] = {}
                    cambios['celdas']  = celdas
                    cambios['ausente'] = cambios['total'] < 5
                    cambios['revisar'] = max(cambios['diferencias']) >= 40
                    resultados[legajo][practico] = cambios
        
        print('Verificando cuadernos duplicados...', archivos)
    return resultados

r = analizar_cambios(['tp1','tp2'])
alumnos = cargar_alumos()
pp.pprint(r)

guardar_alumnos(alumnos, asistencias=r)

Analizando /Users/adibattista/Documents/GitHub/lab4/practicos/tp1/ejercicio.ipynb:
Verificando cuadernos duplicados... ['ejercicio.ipynb']
Analizando /Users/adibattista/Documents/GitHub/lab4/practicos/tp2/Ejercicio.ipynb:
Verificando cuadernos duplicados... ['Ejercicio.ipynb']
{'47121': {'tp1': {'ausente': False,
                   'celdas': 2,
                   'diferencias': [23, 15, 0],
                   'revisar': False,
                   'total': 38},
           'tp2': {'ausente': False,
                   'celdas': 4,
                   'diferencias': [15, 9, 19, 33],
                   'revisar': False,
                   'total': 76}},
 '47417': {'tp1': {'ausente': False,
                   'celdas': 2,
                   'diferencias': [13, 17, 0, 0],
                   'revisar': False,
                   'total': 30},
           'tp2': {'ausente': False,
                   'celdas': 4,
                   'diferencias': [14, 9, 25, 33, 0],
                   'revisar': Fal